In [ ]:
## Here will be the augumentation
# Instead of PIL lib, cv is going to be used here

In [1]:
import os
import cv2

In [2]:
# Input directory: 

base_images_dir = "Images/Resized_Images/images"
base_masks_dir = "Images/Resized_Images/masks"


In [3]:
# Read files - Images and masks

base_images = {}
base_masks = {}

for img_file in os.listdir(base_images_dir):
    if img_file.endswith(".png"):
        # img_num = img_file.split("_")[1] # returns 001.png
        img_num = os.path.splitext(img_file)[0].split("_")[1] # returns 001 -> correct way

        base_images[img_num] = cv2.imread(os.path.join(base_images_dir, img_file))
        mask_file = os.path.join(base_masks_dir, f"label_{img_num}.png")
        mask = cv2.imread(mask_file, cv2.IMREAD_UNCHANGED)

        # Convert mask from 3 channels to 1 channel - needed further for correct augumentation 
        # RGB -> Grayscale conversion
        if mask.ndim == 3:
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
           
        base_masks[img_num] = mask

print(f"{len(base_images)} images and {len(base_masks)} masks have been loaded")

28 images and 28 masks have been loaded


In [46]:
# check if the mask has correct values
print(base_masks['003'])
import numpy as np

unique_values = np.unique(base_masks['003'])
print(unique_values)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[ 0 38]


In [4]:
# Output directory: 

output_images = "Images/Augumented_Images/images"
output_masks = "Images/Augumented_Images/masks"

In [48]:
# Augumentation 01 - Rotating images and masks

import cv2
from albumentations import Compose, Affine
import os

os.makedirs(output_images, exist_ok=True)
os.makedirs(output_masks, exist_ok=True)

# Rotation angles
angles = [10, 30, 60, 90]

for img_num, img in base_images.items():
    mask = base_masks[img_num]

    for angle in angles:
        rot = Compose([Affine(rotate=angle, p=1.0)], p=1.0)
        aug = rot(image=img, mask=mask)
        img_aug = aug['image']
        mask_aug = aug['mask']

        # Save with correct number and nametag
        cv2.imwrite(os.path.join(output_images, f"img_{img_num}_Rotated{angle}Deg.png"), img_aug)
        cv2.imwrite(os.path.join(output_masks, f"label_{img_num}_Rotated{angle}Deg.png"), mask_aug)
        

In [6]:
# Augumentation 02 - Flipping images and masks

import albumentations as A
from albumentations import Compose, Affine
import matplotlib.pyplot as plt
import cv2

os.makedirs(output_images, exist_ok=True)
os.makedirs(output_masks, exist_ok=True)

# Flipping horizontally
transform = Compose([
    A.HorizontalFlip(p=1.0),
])

# Flipping horizontally

for img_num, img in base_images.items():
    mask = base_masks[img_num]

    transformed = transform(image=img, mask = mask)

    img_aug = transformed["image"]
    mask_aug = transformed["mask"]

    # Save with correct number and nametag
    cv2.imwrite(os.path.join(output_images, f"img_{img_num}_Flipped_H.png"), img_aug)
    cv2.imwrite(os.path.join(output_masks, f"label_{img_num}_Flipped_H.png"), mask_aug)

In [7]:
# Augumentation 03 - Saturation adjustment for images and masks
# Augumentation 04 - Contrast adjustment for images and masks
# Augumentation 05 - Edge detection for images and masks
# Augumentation 06 - Sharpening adjustment for images and masks
# Augumentation 07 - Blurring adjustment for images and masks

# Fotometric augumentations are going to be done in the end on the augumented images folder itself with randomness

import os
import cv2
import albumentations as A
import random

output_images = "Images/Augumented_Images/images"
output_masks = "Images/Augumented_Images/masks"

os.makedirs(output_images, exist_ok=True)
os.makedirs(output_masks, exist_ok=True)

# Fotometric augumentations definition
augmentations = {
    "Bright": A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
    "Saturation": A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=30, val_shift_limit=0, p=1.0),
    "Sharpen": A.Sharpen(alpha=(0.2,0.5), lightness=(0.5,1.0), p=1.0),
    "Blur": A.Blur(blur_limit=3, p=1.0)
}

# Percentage of images that are going to be augumented
probability = 0.5

for img_file in os.listdir(output_images):
    if not img_file.endswith(".png"):
        continue

    img_path = os.path.join(output_images, img_file)
    img = cv2.imread(img_path)

    mask_file = img_file.replace("img_", "label_")
    mask_path = os.path.join(output_masks, mask_file)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

    for aug_name, aug_transform in augmentations.items():
        if random.random() > probability:
            continue  # Pick augumentations randomly, to not make the dataset seem fake.

        # augmentation only for the image
        transformed = aug_transform(image=img)
        img_aug = transformed["image"]

        # New file name with type of used augumentation
        base_name = os.path.splitext(img_file)[0]
        img_new_name = f"{base_name}_{aug_name}.png"
        mask_new_name = f"{os.path.splitext(mask_file)[0]}_{aug_name}.png"

        # Saving
        cv2.imwrite(os.path.join(output_images, img_new_name), img_aug)
        cv2.imwrite(os.path.join(output_masks, mask_new_name), mask)